<a href="https://colab.research.google.com/github/Nickguild1993/Business_Py_Explorations/blob/main/pivot_tables_sales_funnel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# PBP work

In [4]:
url = "https://raw.githubusercontent.com/Nickguild1993/Py_EDA_walks/main/sales-funnel.csv"
df = pd.read_csv(url)
df.head(5)

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [5]:
# define "status" column as a category & set order

df["Status"] = df["Status"].astype("category")
df["Status"].cat.set_categories(["won", "pending", "presented", "declined"], inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [7]:
# Basic pivot table

pd.pivot_table(df, index = ["Name"])[:5]

,Account,Price,Quantity
Name,,,
Barton LLC,740150,35000,1.0
"Fritsch, Russel and Anderson",737550,35000,1.0
Herman LLC,141962,65000,2.0
Jerde-Hilpert,412290,5000,2.0
"Kassulke, Ondricka and Metz",307599,7000,3.0


In [10]:
pd.pivot_table(df, index = ["Manager", "Rep"])

Account         Price  Quantity
Manager       Rep                                            
Debra Henley  Craig Booker   720237.0  20000.000000  1.250000
              Daniel Hilton  194874.0  38333.333333  1.666667
              John Smith     576220.0  20000.000000  1.500000
Fred Anderson Cedric Moss    196016.5  27500.000000  1.250000
              Wendy Yule     614061.5  44250.000000  3.000000

In [12]:
# define the columns we want w/ values argument

pd.pivot_table(df, index =["Manager", "Rep"], values = ["Price"]).round(2)

Price
Manager       Rep                    
Debra Henley  Craig Booker   20000.00
              Daniel Hilton  38333.33
              John Smith     20000.00
Fred Anderson Cedric Moss    27500.00
              Wendy Yule     44250.00

Count and Sum - > add then w/ aggfunc = np.sum

In [13]:
pd.pivot_table(df, index = ["Manager", "Rep"], values = ["Price"], aggfunc = np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

aggfunc continued

In [14]:
pd.pivot_table(df, index =["Manager", "Rep"],
               values = ["Price"],
               aggfunc = [np.mean, len])

mean   len
                                    Price Price
Manager       Rep                              
Debra Henley  Craig Booker   20000.000000     4
              Daniel Hilton  38333.333333     3
              John Smith     20000.000000     2
Fred Anderson Cedric Moss    27500.000000     4
              Wendy Yule     44250.000000     4

using optional `columns` arg

In [16]:
pd.pivot_table(df, index = ["Manager", "Rep"],
               values = ["Price"],
               columns = ["Product"],
               aggfunc = [np.sum], fill_value = 0)

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

In [18]:
pd.pivot_table(df, index = ["Manager", "Rep"],
               values = ["Price", "Quantity"],
               columns = ["Product"],
               aggfunc = [np.sum], fill_value = 0)

sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

Can move items to the index to get a different visual representation. removing `product` from columns and add to the index

In [19]:
pd.pivot_table(df, index = ["Manager", "Rep", "Product"],
               values = ["Price"],
               aggfunc = [np.sum], fill_value = 0)

sum
                                          Price
Manager       Rep           Product            
Debra Henley  Craig Booker  CPU           65000
                            Maintenance    5000
                            Software      10000
              Daniel Hilton CPU          105000
                            Software      10000
              John Smith    CPU           35000
                            Maintenance    5000
Fred Anderson Cedric Moss   CPU           95000
                            Maintenance    5000
                            Software      10000
              Wendy Yule    CPU          165000
                            Maintenance    7000
                            Monitor        5000

For totals, add arg `margins = True`

In [22]:
pd.pivot_table(df, index = ["Manager", "Rep", "Product"],
               values = ["Price", "Quantity"],
               aggfunc = [np.sum, np.mean], margins = True,
               fill_value= 0)

sum                   mean  \
                                          Price Quantity         Price   
Manager       Rep           Product                                      
Debra Henley  Craig Booker  CPU           65000        2  32500.000000   
                            Maintenance    5000        2   5000.000000   
                            Software      10000        1  10000.000000   
              Daniel Hilton CPU          105000        4  52500.000000   
                            Software      10000        1  10000.000000   
              John Smith    CPU           35000        1  35000.000000   
                            Maintenance    5000        2   5000.000000   
Fred Anderson Cedric Moss   CPU           95000        3  47500.000000   
                            Maintenance    5000        1   5000.000000   
                            Software      10000        1  10000.000000   
              Wendy Yule    CPU          165000        7  82500.000000   
                            Maintenance    7000        3   7000.000000   
                            Monitor        5000        2   5000.000000   
All                                      522000       30  30705.882353   

                                                   
                                         Quantity  
Manager       Rep           Product                
Debra Henley  Craig Booker  CPU          1.000000  
                            Maintenance  2.000000  
                            Software     1.000000  
              Daniel Hilton CPU          2.000000  
                            Software     1.000000  
              John Smith    CPU          1.000000  
                            Maintenance  2.000000  
Fred Anderson Cedric Moss   CPU          1.500000  
                            Maintenance  1.000000  
                            Software     1.000000  
              Wendy Yule    CPU          3.500000  
                            Maintenance  3.000000  
                            Monitor      2.000000  
All                                      1.764706

In [23]:
pd.pivot_table(df, index = ["Manager", "Status"],
               values = ["Price"],
               aggfunc = [np.sum],
               margins = True,
               fill_value = 0)

sum
                          Price
Manager       Status           
Debra Henley  won         65000
              pending     50000
              presented   50000
              declined    70000
Fred Anderson won        172000
              pending      5000
              presented   45000
              declined    65000
All                      522000

passing a dictionary into the pivot_table `aggfunc` argument 

In [29]:
df.head(1)

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented


In [32]:
# aggfunc = {"Quantity" : len, "Price" : np.sum}

table = pd.pivot_table(df, index = ["Manager", "Status"],
               columns = ["Product"],
               values = ["Quantity", "Price"],
               aggfunc = {"Quantity" : len, "Price" : [np.sum, np.mean]},
               fill_value =0)
table

Price                                       \
                          mean                                  sum   
Product                    CPU Maintenance Monitor Software     CPU   
Manager       Status                                                  
Debra Henley  won        65000           0       0        0   65000   
              pending    40000        5000       0        0   40000   
              presented  30000           0       0    10000   30000   
              declined   35000           0       0        0   70000   
Fred Anderson won        82500        7000       0        0  165000   
              pending        0        5000       0        0       0   
              presented  30000           0    5000    10000   30000   
              declined   65000           0       0        0   65000   

                                                     Quantity              \
                                                          len               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Debra Henley  won                 0       0        0        1           0   
              pending         10000       0        0        1           2   
              presented           0       0    20000        1           0   
              declined            0       0        0        2           0   
Fred Anderson won              7000       0        0        2           1   
              pending          5000       0        0        0           1   
              presented           0    5000    10000        1           0   
              declined            0       0        0        1           0   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  won             0        0  
              pending         0        0  
              presented       0        2  
              declined        0        0  
Fred Anderson won             0        0  
              pending         0        0  
              presented       1        1  
              declined        0        0

### Once you have the pivot table, its in a dataframe. So it can be filtered.

check the quotations for this - .query() will be ' ' for the string and then " " wrapping the filtered value (assuming it's a string)

In [36]:
table.query('Manager == ["Debra Henley"]')

Price                                                  \
                         mean                                 sum               
Product                   CPU Maintenance Monitor Software    CPU Maintenance   
Manager      Status                                                             
Debra Henley won        65000           0       0        0  65000           0   
             pending    40000        5000       0        0  40000       10000   
             presented  30000           0       0    10000  30000           0   
             declined   35000           0       0        0  70000           0   

                                        Quantity                               
                                             len                               
Product                Monitor Software      CPU Maintenance Monitor Software  
Manager      Status                                                            
Debra Henley won             0        0        1           0       0        0  
             pending         0        0        1           2       0        0  
             presented       0    20000        1           0       0        2  
             declined        0        0        2           0       0        0

In [39]:
table.query('Status == ["pending", "won"]')

Price                                                   \
                        mean                                  sum               
Product                  CPU Maintenance Monitor Software     CPU Maintenance   
Manager       Status                                                            
Debra Henley  won      65000           0       0        0   65000           0   
              pending  40000        5000       0        0   40000       10000   
Fred Anderson won      82500        7000       0        0  165000        7000   
              pending      0        5000       0        0       0        5000   

                                       Quantity                               
                                            len                               
Product               Monitor Software      CPU Maintenance Monitor Software  
Manager       Status                                                          
Debra Henley  won           0        0        1           0       0        0  
              pending       0        0        1           2       0        0  
Fred Anderson won           0        0        2           1       0        0  
              pending       0        0        0           1       0        0